In [6]:
from pathlib import Path
from beak.remote import MMseqsSearch

search = MMseqsSearch(host="shr-zion.stanford.edu", 
                      user="mbolivas",
                      key_path="~/.ssh/shr-zion")

# List available presets
search.list_presets()

,preset,description,parameters
0,default,Default MMseqs2 sensitivity (balanced speed/se...,"s=5.7, e=0.001"
1,fast,Fast search for close homologs,"s=4.0, e=0.001"
2,sensitive,More sensitive search (slower),"s=7.5, e=0.001"
3,exhaustive,Exhaustive search for remote homologs,"s=8.5, e=10, max_seqs=10000, min_seq_id=0.15, ..."
4,very_sensitive,Very sensitive (good for distant homologs),"s=8.0, e=0.01, max_seqs=1000"


In [7]:
test_fasta = Path("hAcyP2_exhaustive_query.fasta")
test_fasta.write_text(""">hAcyP2
MSTAQSLKSVDYEVFGRVQGVCFRMYTEDEARKIGVVGWVKNTSKGTVTGQVQGPEDKVNSMKSWLSKVGSPSSRIDRTNFSNEKTISKLEYSNFSIRY
""")


108

In [8]:
# Use a preset
job_id = search.submit(
    query_file="hAcyP2_expansive_query.fasta",
    database="swissprot",
    preset="fast",
    job_name="hAcyP2_exhaustive_search",
    threads=20
)

Using preset: fast - Fast search for close homologs
Created remote directory: /home/mbolivas/beak_jobs/00a917ec
Uploading query file...
✓ Job submitted: hAcyP2_exhaustive_search (ID: 00a917ec)
  Database: swissprot (/srv/protein_sequence_databases/swissprot)
  PID: 207784


In [9]:
search.list_jobs()

,job_id,name,status,database,submitted,runtime
2,00a917ec,hAcyP2_exhaustive_search,RUNNING,swissprot,2025-11-11T17:20:06.728306,0:00:09
1,0500f5c3,hAcyP2_exhaustive_search,COMPLETED,uniref90,2025-11-11T17:18:31.717534,0:01:44
0,5e1878a0,hAcyP2_exhaustive_search,COMPLETED,uniref90,2025-11-10T10:52:08.343932,"1 day, 6:28:07"


In [11]:
search.download(job_id)

✓ Downloaded results to 00a917ec_results.m8
✓ Downloaded log to 00a917ec_log.txt


PosixPath('00a917ec_results.m8')